In [1]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = '/content/drive/MyDrive/CatClassifier/train'

In [3]:
train_datagen = ImageDataGenerator(validation_split=0.2,rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='training',
    class_mode='binary',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)

Found 20001 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [6]:
def inception(input_tensor, filter_channels):
    filter_1x1, filter_3x3_B, filter_3x3, filter_5x5_B, filter_5x5, max_pool_A = filter_channels
    # filter_3x3_B : 3x3 컨볼루션 전(Before)의 1x1 컨볼루션의 필터 갯수
    # filter_5x5_B : 5x5 컨볼루션 전(Before)의 1x1 컨볼루션의 필터 갯수
    # max_pool_A : 3x3 Max Pooling층 이후(After)의 1x1 컨볼루션의 필터 갯수
    branch_1 = keras.layers.Conv2D(filter_1x1, (1, 1), strides=1, padding='same', activation='relu')(input_tensor)
    
    branch_2 = keras.layers.Conv2D(filter_3x3_B, (1, 1), strides=1, padding='same', activation='relu')(input_tensor)
    branch_2 = keras.layers.Conv2D(filter_3x3, (3, 3), strides=1, padding='same', activation='relu')(branch_2)

    branch_3 = keras.layers.Conv2D(filter_5x5_B, (1, 1), strides=1, padding='same', activation='relu')(input_tensor)
    branch_3 = keras.layers.Conv2D(filter_5x5, (5, 5), strides=1, padding='same', activation='relu')(branch_3)
    
    branch_4 = keras.layers.MaxPooling2D((3, 3), strides=1, padding='same')(input_tensor)
    branch_4 = keras.layers.Conv2D(max_pool_A, (1, 1), strides=1, padding='same', activation='relu')(branch_4)
    
    DepthConcat = keras.layers.Concatenate()([branch_1, branch_2, branch_3, branch_4])
    
    return DepthConcat

In [9]:
def GoogLeNet(model_input, classes=2):
    conv_1 = layers.Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(model_input)
    pool_1 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(conv_1) 
    LRN_1 = layers.Lambda(tf.nn.local_response_normalization)(pool_1) # (56, 56, 64)
    
    conv_2 = layers.Conv2D(64, (1, 1), strides=1, padding='valid', activation='relu')(LRN_1) # (56, 56, 64)
    conv_3 = layers.Conv2D(192, (3, 3), strides=1, padding='same', activation='relu')(conv_2) # (56, 56, 192)
    LRN_2 = layers.Lambda(tf.nn.local_response_normalization)(conv_3) # (56, 56, 192)
    pool_2 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(LRN_2) # (28, 28, 192)
    
    inception_3a = inception(pool_2, [64, 96, 128, 16, 32, 32]) # (28, 28, 256)
    inception_3b = inception(inception_3a, [128, 128, 192, 32, 96, 64]) # (28, 28, 480)
    
    pool_3 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(inception_3b) # (14, 14, 480)
    
    inception_4a = inception(pool_3, [192, 96, 208, 16, 48, 64]) # (14, 14, 512)
    inception_4b = inception(inception_4a, [160, 112, 224, 24, 64, 64]) # (14, 14, 512)
    inception_4c = inception(inception_4b, [128, 128, 256, 24, 64, 64]) # (14, 14, 512)
    inception_4d = inception(inception_4c, [112, 144, 288, 32, 64, 64]) # (14, 14, 528)
    inception_4e = inception(inception_4d, [256, 160, 320, 32, 128, 128]) # (14, 14, 832)
    
    pool_4 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(inception_4e) # (7, 7, 832)
    
    inception_5a = inception(pool_4, [256, 160, 320, 32, 128, 128]) # (7, 7, 832)
    inception_5b = inception(inception_5a, [384, 192, 384, 48, 128, 128]) # (7, 7, 1024)
    
    avg_pool = layers.GlobalAveragePooling2D()(inception_5b)
    dropout = layers.Dropout(0.4)(avg_pool)
    
    linear = layers.Dense(1000, activation='relu')(dropout)
    
    model_output = layers.Dense(classes, activation='softmax', name='main_classifier')(linear) # 'softmax'
    
    #Auxiliary Classifier

    auxiliary_4a = layers.AveragePooling2D((5, 5), strides=3, padding='valid')(inception_4a)
    auxiliary_4a = layers.Conv2D(128, (1, 1), strides=1, padding='same', activation='relu')(auxiliary_4a)
    auxiliary_4a = layers.Flatten()(auxiliary_4a)
    auxiliary_4a = layers.Dense(1024, activation='relu')(auxiliary_4a)
    auxiliary_4a = layers.Dropout(0.7)(auxiliary_4a)
    auxiliary_4a = layers.Dense(classes, activation='softmax', name='auxiliary_4a')(auxiliary_4a)
    
    auxiliary_4d = layers.AveragePooling2D((5, 5), strides=3, padding='valid')(inception_4d)
    auxiliary_4d = layers.Conv2D(128, (1, 1), strides=1, padding='same', activation='relu')(auxiliary_4d)
    auxiliary_4d = layers.Flatten()(auxiliary_4d)
    auxiliary_4d = layers.Dense(1024, activation='relu')(auxiliary_4d)
    auxiliary_4d = layers.Dropout(0.7)(auxiliary_4d)
    auxiliary_4d = layers.Dense(classes, activation='softmax', name='auxiliary_4d')(auxiliary_4d)
    
    
    model = keras.Model(model_input, [model_output, auxiliary_4a, auxiliary_4d])
    
    return model

In [10]:
input_shape = (224,224,3)
model_input = layers.Input(shape=input_shape)
model = GoogLeNet(model_input)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_59 (Conv2D)              (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_59[0][0]                  
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 56, 56, 64)   0           max_pooling2d_13[0][0]           
______________________________________________________________________________________________

In [22]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, decay = 0.0005),
              loss={'main_classifier' : 'sparse_categorical_crossentropy',
                    'auxiliary_4a' : 'sparse_categorical_crossentropy',
                    'auxiliary_4d' : 'sparse_categorical_crossentropy'},
                loss_weights={'main_classifier' : 1.0,
                              'auxiliary_4a' : 0.3,
                              'auxiliary_4d' : 0.3},
                metrics=['accuracy'])

In [23]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [24]:
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/CatClassifier/GoogLeNet',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',)

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose = 1, mode= 'auto')

In [25]:
history = model.fit(train_generator,
                 steps_per_epoch=100, 
                 epochs = 100 , 
                 validation_data=validation_generator ,
                 validation_steps=50 ,
                 verbose=1,
                 callbacks=[checkpoint, early])

Epoch 1/100
100/100 [==============================] - 1827s 18s/step - loss: 1.1088 - main_classifier_loss: 0.6930 - auxiliary_4a_loss: 0.6928 - auxiliary_4d_loss: 0.6930 - main_classifier_accuracy: 0.5054 - auxiliary_4a_accuracy: 0.5057 - auxiliary_4d_accuracy: 0.5110 - val_loss: 1.1087 - val_main_classifier_loss: 0.6933 - val_auxiliary_4a_loss: 0.6917 - val_auxiliary_4d_loss: 0.6927 - val_main_classifier_accuracy: 0.4762 - val_auxiliary_4a_accuracy: 0.5562 - val_auxiliary_4d_accuracy: 0.5506

Epoch 00001: val_loss improved from inf to 1.10868, saving model to /content/drive/MyDrive/CatClassifier/GoogLeNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/GoogLeNet/assets
Epoch 2/100
100/100 [==============================] - 1553s 16s/step - loss: 1.1084 - main_classifier_loss: 0.6932 - auxiliary_4a_loss: 0.6914 - auxiliary_4d_loss: 0.6928 - main_classifier_accuracy: 0.4954 - auxiliary_4a_accuracy: 0.5339 - auxiliary_4d_accuracy: 0.5178 - val_loss: 1.1086 - val

In [26]:
model.save('/content/drive/MyDrive/CatClassifier/GoogLeNet/additional train (1)')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/GoogLeNet/additional train (1)/assets


In [27]:
model.evaluate(train_generator)

626/626 [==============================] - 85s 135ms/step - loss: 1.0063 - main_classifier_loss: 0.6269 - auxiliary_4a_loss: 0.6388 - auxiliary_4d_loss: 0.6259 - main_classifier_accuracy: 0.6505 - auxiliary_4a_accuracy: 0.6527 - auxiliary_4d_accuracy: 0.6555


[1.0062897205352783,
 0.6268638968467712,
 0.6388272047042847,
 0.6259243488311768,
 0.6504674553871155,
 0.6527173519134521,
 0.6555172204971313]